In [0]:
%python


%md
1127 - User Purchase _Platform_

Write a solution to find the total number of users and the total amount spent using the mobile only, the desktop only, and both mobile and desktop together for each date.

Return the result table in any order.

The result format is in the following example.


Explanation: 
On 2019-07-01, user 1 purchased using both desktop and mobile, user 2 purchased using mobile only and user 3 purchased using desktop only.
On 2019-07-02, user 2 purchased using mobile only, user 3 purchased using desktop only and no one purchased using both platforms.

https://leetcode.ca/2018-12-31-1127-User-Purchase-Platform/

In [0]:
create schema leet

In [0]:
CREATE TABLE leet.user_spend (
    user_id INT,
    spend_date DATE,
    platform STRING,
    amount INT
)
USING DELTA;

-- Insert data into the Delta table
INSERT INTO leet.user_spend VALUES
    (1, '2019-07-01', 'mobile', 100),
    (1, '2019-07-01', 'desktop', 100),
    (2, '2019-07-01', 'mobile', 100),
    (2, '2019-07-02', 'mobile', 100),
    (3, '2019-07-01', 'desktop', 100),
    (3, '2019-07-02', 'desktop', 100);

num_affected_rows,num_inserted_rows
6,6


In [0]:
select * from leet.user_spend

user_id,spend_date,platform,amount
1,2019-07-01,mobile,100
1,2019-07-01,desktop,100
2,2019-07-01,mobile,100
2,2019-07-02,mobile,100
3,2019-07-01,desktop,100
3,2019-07-02,desktop,100


In [0]:
describe leet.user_spend

col_name,data_type,comment
user_id,int,null
spend_date,date,null
platform,string,null
amount,int,null


- Group by and aggragation of string columns

In [0]:
select 
  spend_date,
  platform,
  amount,
  array_join(collect_list(user_id),',')
from leet.user_spend
group by
  spend_date,
  platform,
  amount

spend_date,platform,amount,"array_join(collect_list(user_id), ,)"
2019-07-02,desktop,100,3
2019-07-01,mobile,100,"1,2"
2019-07-02,mobile,100,2
2019-07-01,desktop,100,"1,3"


In [0]:
with c1 as (
  select 
  user_id,
  spend_date,
  amount,
  array_join(collect_list(platform),',')
from leet.user_spend
group by
user_id,
  spend_date,
  amount
)
select * from c1

user_id,spend_date,amount,"array_join(collect_list(platform), ,)"
2,2019-07-01,100,mobile
3,2019-07-01,100,desktop
2,2019-07-02,100,mobile
3,2019-07-02,100,desktop
1,2019-07-01,100,"mobile,desktop"


In [0]:
with cu as (
  select distinct  spend_date, 'both' as platform from leet.user_spend
  union
  select distinct  spend_date, 'desktop' as platform from leet.user_spend
  union 
  select distinct  spend_date, 'mobile' as platform from leet.user_spend
)
select * from cu

spend_date,platform
2019-07-01,both
2019-07-02,both
2019-07-01,desktop
2019-07-02,desktop
2019-07-01,mobile
2019-07-02,mobile


In [0]:

with cu as
  (select distinct spend_date,
                   'both' as platform
   from leet.user_spend
   union select distinct spend_date,
                         'desktop' as platform
   from leet.user_spend
   union select distinct spend_date,
                         'mobile' as platform
   from leet.user_spend),
     c1 as
  (select user_id,
          spend_date,
          sum(amount) amount1,
          count(platform) cc
   from leet.user_spend
   group by user_id,
            spend_date),
     c2 as
  (select distinct us.user_id,
                   us.spend_date,
                   amount1,
                   platform,
                   cc,
                   iff(cc=1, us.platform, 'both') platf
   from leet.user_spend us
   left join c1 on us.user_id=c1.user_id
   and us.spend_date=c1.spend_date)
select cu.spend_date,
       cu.platform,
       ifnull(c2.amount1, 0),
       approx_count_distinct(user_id)
from cu
left join c2 on cu.spend_date=c2.spend_date
and cu.platform=c2.platf
group by cu.spend_date,
         cu.platform,
         c2.amount1



spend_date,platform,"ifnull(amount1, 0)",approx_count_distinct(user_id)
2019-07-01,both,200,1
2019-07-02,both,0,0
2019-07-01,desktop,100,1
2019-07-02,desktop,100,1
2019-07-01,mobile,100,1
2019-07-02,mobile,100,1
